In [207]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_predict
from sklearn import metrics

In [208]:
# Load data from csv
file_path_1 = "C:/Users/eliez/AnacondaProjects/Gender_Classifier/"
name_of_file_read = "nomes_masculino_raw.txt"
file_to_ready = file_path_1 + name_of_file_read

# read and show the raw 
headers = ["name"]
dataM = pd.read_csv(file_to_ready ,engine='python', error_bad_lines = False , encoding='utf8' ,names = headers)

# Add Gender type and Capital first letter
dataM['gender'] = 'M'
dataM['name'] = dataM['name'].str.capitalize()

#Clean duplicate values
dup = dataM.duplicated(subset=['name']).any()
print(dup)
dataM.drop_duplicates(subset=['name'], inplace=True)

# shape of the DataFrame
print("shape : ", dataM.shape)
dataM.head(3)

True
shape :  (1362, 2)


,name,gender
0,Aarão,M
1,Abdias,M
2,Abel,M


In [209]:
# Load data from csv
file_path_1 = "C:/Users/eliez/AnacondaProjects/Gender_Classifier/"
name_of_file_read = "nomes_feminino_raw.txt"
file_to_ready = file_path_1 + name_of_file_read

# read and show the raw data
headers = ["name"]
dataF = pd.read_csv(file_to_ready ,engine='python', error_bad_lines = False , encoding='utf8' ,names = headers)

# Add Gender type and Capital first letter
dataF['gender'] = 'F'
dataF['name'] = dataF['name'].str.capitalize()

#Clean duplicate values
dup = dataF.duplicated(subset=['name']).any()
print(dup)
dataF.drop_duplicates(subset=['name'], inplace=True)

# shape of the DataFrame
print("shape : ", dataF.shape)
dataF.head(3)

True
shape :  (794, 2)


,name,gender
0,Lisandra,F
1,Melanie,F
2,Mila,F


In [210]:
# Get unisex names (i), names in both dataframes
dataBoth = pd.merge(dataM, dataF, how='inner', on=['name', 'name'])
cols = ['gender_x','gender_y']
dataBoth.drop(cols, axis=1 , inplace=True)
dataBoth['gender'] = 'U'
print("shape : ", dataBoth.shape)
dataBoth.head(3)

shape :  (2, 2)


,name,gender
0,Dagmar,U
1,Nadir,U


In [211]:
# Append all datas (Male, Female and Unisex)
dataFinal = dataM.append([dataF,dataBoth], ignore_index=True)
dataFinal.shape

(2158, 2)

In [212]:
# Naives Bayes process
namesVec = dataFinal["name"].values
genderVec = dataFinal["gender"].values

In [213]:
namesVec

array(['Aarão', 'Abdias', 'Abel', ..., 'Antonela', 'Dagmar', 'Nadir'],
      dtype=object)

In [214]:
genderVec

array(['M', 'M', 'M', ..., 'F', 'U', 'U'], dtype=object)

In [215]:
# Modeling
vectorizer = CountVectorizer(analyzer = "char")
freq = vectorizer.fit_transform(namesVec)
modelo = MultinomialNB()
modelo.fit(freq, genderVec)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [216]:
# Cross validatiom. For each 24 trainings tests 1 
resultados = cross_val_predict(modelo, freq, genderVec, cv = 25)
genders = ["M", "F", "U"]
print(metrics.classification_report(genderVec, resultados, genders))

C:\Programas\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=25.
  % (min_groups, self.n_splits)), Warning)


             precision    recall  f1-score   support

          M       0.77      0.82      0.79      1362
          F       0.65      0.58      0.62       794
          U       0.00      0.00      0.00         2

avg / total       0.73      0.73      0.73      2158



C:\Programas\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [217]:
# Testing
testes = ["Eliezer",
          "Luigi",
          "Ana",
          "Pedrinho",
          "João",
          "Mariazinha"]

freq_testes = vectorizer.transform(testes)
resultados = modelo.predict(freq_testes)

data_frame = pd.DataFrame(data = { "name": testes, "gender": resultados })
data_frame

,gender,name
0,M,Eliezer
1,M,Luigi
2,F,Ana
3,M,Pedrinho
4,M,João
5,F,Mariazinha
